In [1]:
%%writefile src/Rpc.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract A {
    string Athing;
    constructor() { }
    receive() external payable {}
    fallback() external payable {}
    function getBalance() public view returns(uint) {
        return (address(this).balance);
    }
    function setAthing(string memory s) public {
        Athing = s;
    }
    function getAthing() public view returns(string memory) {
        return(Athing);
    }
}

contract B {
    string Bthing;
    constructor() { }
    receive() external payable {}
    fallback() external payable {}
    function getBalance() public view returns(uint) {
        return (address(this).balance);
    }
    function setBthing(string memory s) public {
        Bthing = s;
    }
    function getBthing() public view returns(string memory) {
        return(Bthing);
    }
}
contract Rpc {
    
    mapping (uint => string) private rpc;
    uint balance;
    string res;
    address owner;
    A a1 = new A();
    B b1 = new B();

    event Show(address _address);
    event PrintLog(string str);
    constructor() payable {
        owner = msg.sender;
        balance = 0;
        rpc[0] = "rock";
        rpc[1] = "scissors";
        rpc[2] = "paper";
    }   

    fallback () external {
        emit PrintLog("fallback called");
    }

    function deposit(uint amount) public payable  {
        require(amount==msg.value);
        balance += amount;
    }
    // 베팅 = deposit

    function setA() public payable {
        require(msg.value == 1000);
        a1.setAthing(rpc[uint8(uint256(keccak256(abi.encodePacked(block.timestamp, block.difficulty)))%3)]);
        balance += msg.value;
        emit Show(address(a1));
    }

    function setB(uint num) public payable {
        require(msg.value == 1000);
        b1.setBthing(rpc[num]);
        balance += msg.value;
        emit Show(address(b1));

    }

    function play() public payable {
        string memory Athing = a1.getAthing();
        string memory Bthing = b1.getBthing();
        address a1add = address(a1);
        address b1add = address(b1);
        
        if (keccak256(bytes(Athing)) == keccak256(bytes(Bthing))) {
            res = "tie";
            require(address(this).balance >= 2000);
            payable(a1add).transfer(1000);
            payable(b1add).transfer(1000);
            balance -= 2000;

       
        }
      
        else if ( (keccak256(bytes(Athing)) == keccak256(bytes(rpc[0])) && keccak256(bytes(Bthing)) == keccak256(bytes(rpc[1]))) 
        || 
        ( keccak256(bytes(Athing)) == keccak256(bytes(rpc[1])) && keccak256(bytes(Bthing)) == keccak256(bytes(rpc[2]))) 
        || 
        ( keccak256(bytes(Athing)) == keccak256(bytes(rpc[2])) && keccak256(bytes(Bthing)) == keccak256(bytes(rpc[0])))) {
            res = "A wins";
            require(address(this).balance >= 2000);
            payable(a1add).transfer(address(this).balance);
            balance -= 2000;

        }

        else {
            res = "B wins";
            require(address(this).balance >= 2000);
            payable(b1add).transfer(address(this).balance);
            balance -= 2000;


        }
    }

    function getMatchResult() public view returns (string memory) {
        return res;
    }

    function getBalance() public view returns(uint, uint) {
        return (balance, address(this).balance);
    }

    function getABalance() public view returns(uint) {
        return a1.getBalance();
    }
    function getBbalance() public view returns(uint) {
        return b1.getBalance();
    }
    function sendA() public payable {
        payable(address(a1)).transfer(1000);
    }
}

Writing src/Rpc.sol


In [2]:
!solc --optimize --combined-json abi,bin src/Rpc.sol > src/Rpc.json

In [29]:
%%writefile src/Rpc.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Rpc.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/C1C2.sol:C2"].abi);
var _abiArray=_json.contracts["src/Rpc.sol:Rpc"].abi;
//var _bin=_json.contracts.sHello2.bin;
var _bin="0x"+_json.contracts["src/Rpc.sol:Rpc"].bin;

//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 3000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
        //.then(function(newContractInstance){
        //    console.log(newContractInstance.options.address)
        //});
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/Rpc.js


In [30]:
!node src/Rpc.js

Deploying the contract from 0xBE2e9a0e75f45Cc07cA45C4e64bf97deb6F783E5
>>> transactionHash0x854a4c9803f187f59afbfd7d9656eda4acefe9a22c8a57b364321165a2caaaf1
>>> RECEPIT hash: 0x854a4c9803f187f59afbfd7d9656eda4acefe9a22c8a57b364321165a2caaaf1
>>> address:0xAC31d726d15a028d9Ff6122D53E4f1aD7e7C8ba0
---> The contract deployed to: 0xAC31d726d15a028d9Ff6122D53E4f1aD7e7C8ba0


In [31]:
%%writefile src/RpcUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Rpc.json");
var _json=JSON.parse(_str)
//var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
//var _abiArray=JSON.parse(_json.contracts["src/Rpc.sol:Rpc"].abi);
var _abiArray=_json.contracts["src/Rpc.sol:Rpc"].abi;

var Rpc = new web3.eth.Contract(_abiArray, "0xAC31d726d15a028d9Ff6122D53E4f1aD7e7C8ba0");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    await Rpc.methods.setA().send({from: accounts[0],gas:600000, value:1000}, function(err) {
        console.log("setting A...");
    });
    // 0 = "rock", 1 = "scissors", 2 = "paper"
    await Rpc.methods.setB(0).send({from: accounts[0],gas:600000, value:1000}, function(err) {
        console.log("setting B...")
    });
    
    await Rpc.methods.play().send({from: accounts[0],gas:600000}, function(err) {
        console.log("Playing RPC...")
    })
    
    await Rpc.methods.getMatchResult().call().then(function(res) {
        console.log("Match Result: " + res)
    });
    
        
    await Rpc.methods.getABalance().call().then(function(res) {
        console.log("Balance of Contract A: " + res)
    });
    
    await Rpc.methods.getBbalance().call().then(function(res) {
        console.log("Balance of Contract B: " + res)
    });
    
    await Rpc.methods.getBalance().call().then(function(res) {
        console.log("Balance of Contract RPC: " + res[0] + " , " + res[1])
    });
    
}

doIt()

Overwriting src/RpcUse.js


In [32]:
!node src/RpcUse.js

Account: 0xBE2e9a0e75f45Cc07cA45C4e64bf97deb6F783E5
setting A...
setting B...
Playing RPC...
Match Result: tie
Balance of Contract A: 1000
Balance of Contract B: 1000
Balance of Contract RPC: 0 , 0
